<a href="https://colab.research.google.com/github/AK-Github-0/TTS-Finetuning-for-Arabic/blob/main/nb/Oute_TTS_(1B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[TTS Guide](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning)** for instructions and all our notebooks.

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

# Extra installations for Oute
!pip install omegaconf einx
!git clone https://github.com/edwko/OuteTTS
# import os
# os.remove("/content/OuteTTS/outetts/models/gguf_model.py")
# os.remove("/content/OuteTTS/outetts/interface.py")
# os.remove("/content/OuteTTS/outetts/__init__.py")
!pip install pyloudnorm openai-whisper uroman MeCab loguru flatten_dict ffmpy randomname argbind tiktoken
!pip install descript-audio-codec descript-audiotools julius openai-whisper --no-deps
%env UNSLOTH_DISABLE_FAST_GENERATION = 1

In [2]:
!pip install transformers pyarabic
!pip install arabert
!pip install ftfy

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

Thank you to [Etherl](https://huggingface.co/Etherll) for creating this notebook!

In [3]:
from unsloth import FastModel
import torch
max_seq_length = 2048

# Load the OuteTTS model
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Llama-OuteTTS-1.0-1B",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = False,
)

# Let's test with Arabic text directly (no preprocessing needed!)
test_text = "مرحبا بكم في عالم الذكاء الاصطناعي"
# We'll feed Arabic text directly to the model
print(f"Arabic text: {test_text}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.
Arabic text: مرحبا بكم في عالم الذكاء الاصطناعي


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
# Add LoRA adapters for fine-tuning
model = FastModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "v_proj",],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep  

We will use the `MrDragonFox/Elise`, which is designed for training TTS models. Ensure that your dataset follows the required format: **text, audio**, but maintaining the correct structure is essential for optimal training.

In [5]:
from datasets import load_dataset, Audio, Dataset

# Load the dataset
dataset = load_dataset("MohamedRashad/SCC22", split = 'test')
dataset = dataset.cast_column("audio", Audio(sampling_rate=24000))

# For faster testing, use a smaller subset
# Uncomment the next line to use just 50 examples
dataset = dataset.select(range(50))

print(f"Dataset loaded with {len(dataset)} examples")
print(f"Example transcript: {dataset[0]['cleaned_text']}")

Dataset loaded with 50 examples
Example transcript: اترككم مع الحوار حياك الله ابو فيصل شرفتنا ونورتنا حياك الله يسلمك ويبقيك الحمد لله على سلامتك


In [6]:
import torch
from tqdm import tqdm
import io
import tempfile
import sys
sys.path.append('OuteTTS')
import os
import dac
from outetts.version.v3.audio_processor import AudioProcessor
from outetts.version.v3.prompt_processor import PromptProcessor
from outetts.models.config import ModelConfig
import whisper
from outetts.utils.preprocessing import text_normalizations
import soundfile as sf
import numpy as np
import re

class ArabicTTSProcessor:
    def __init__(
            self,
            whisper_model_name: str = "large-v2",
            device: str = None
        ):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Use the OuteTTS model's path - it has native Arabic support!
        model_tokenizer_path = "unsloth/Llama-OuteTTS-1.0-1B"

        # Create ModelConfig for audio processing
        dummy_config = ModelConfig(
            tokenizer_path=model_tokenizer_path,
            device=self.device,
            audio_codec_path=None
        )
        self.audio_processor = AudioProcessor(config=dummy_config)
        self.prompt_processor = PromptProcessor(model_tokenizer_path)

        print(f"Loading Whisper model: {whisper_model_name} on {self.device}")
        self.whisper_model = whisper.load_model(whisper_model_name, device=self.device)
        print("Whisper model loaded.")

    def create_speaker_representation(self, audio_bytes: bytes, transcript: str):
        """Create speaker representation with native Arabic text support"""
        if not audio_bytes or not transcript:
             print("Missing audio bytes or transcript.")
             return None

        # Use Arabic text directly - OuteTTS has native Arabic support!
        print(f"Processing Arabic text: {transcript[:30]}{'...' if len(transcript) > 30 else ''}")

        try:
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp_audio_file:
                tmp_audio_file.write(audio_bytes)
                tmp_audio_file.flush()

                # Force Arabic language recognition for better word timing
                whisper_result = self.whisper_model.transcribe(
                    tmp_audio_file.name,
                    word_timestamps=True,
                    language="ar"
                )

                # Use Arabic transcript directly
                normalized_transcript = text_normalizations(transcript)

                words_with_timings = []
                if whisper_result and 'segments' in whisper_result:
                    for segment in whisper_result['segments']:
                        if 'words' in segment:
                            for word_info in segment['words']:
                                cleaned_word = word_info['word'].strip()
                                if cleaned_word:
                                    # Use Arabic words directly
                                    words_with_timings.append({
                                        'word': cleaned_word,
                                        'start': float(word_info['start']),
                                        'end': float(word_info['end'])
                                    })
                else:
                    print(f"Whisper did not return segments/words.")
                    return None

                if not words_with_timings:
                    print(f"No word timings extracted.")
                    return None

                # Prepare the data dictionary
                speaker_data_dict = {
                    "audio": {"bytes": audio_bytes},
                    "text": normalized_transcript,
                    "words": words_with_timings
                }

                # Create the speaker representation
                v3_speaker = self.audio_processor.create_speaker_from_dict(speaker_data_dict)
                return v3_speaker

        except Exception as e:
            print(f"Error during speaker creation: {str(e)}")
            import traceback
            traceback.print_exc()
            return None

    def process_dataset(self, dataset):
        """Process dataset with native Arabic text support"""
        processed_count = 0
        skipped_count = 0

        for i, item in enumerate(tqdm(dataset, desc="Processing Arabic Dataset")):
            try:
                transcript = item.get('cleaned_text')
                audio_info = item.get('audio')

                if not transcript or not isinstance(transcript, str):
                    print(f"Row {i}: Skipping due to invalid text.")
                    skipped_count += 1
                    continue

                # Extract audio data
                audio_array = audio_info['array']
                buffer = io.BytesIO()
                sf.write(buffer, audio_array.astype(np.float32), audio_info['sampling_rate'], format='WAV', subtype='FLOAT')
                buffer.seek(0)
                audio_bytes = buffer.getvalue()

                # Create speaker representation with native Arabic text
                speaker = self.create_speaker_representation(audio_bytes, transcript)

                if speaker is None:
                    print(f"Row {i}: Failed to create speaker representation.")
                    skipped_count += 1
                    continue

                # Get the training prompt
                prompt = self.prompt_processor.get_training_prompt(speaker)

                # Verify we have a valid prompt (debug output)
                if i == 0:
                    print("First sample prompt (truncated):")
                    print(prompt[:200] + "..." if len(prompt) > 200 else prompt)

                processed_count += 1
                yield prompt

            except KeyboardInterrupt:
                 print("Processing interrupted by user.")
                 break
            except Exception as e:
                print(f"Row {i}: Error processing item: {str(e)}")
                import traceback
                traceback.print_exc()
                skipped_count += 1
                continue

        print(f"Arabic dataset processing finished. Processed: {processed_count}, Skipped: {skipped_count}")

In [7]:
# Initialize the Arabic processor
data_processor = ArabicTTSProcessor(
    whisper_model_name="large-v2"
)

# Process the dataset with native Arabic support
all_prompts = []
print("Starting Arabic dataset processing...")
processed_dataset = data_processor.process_dataset(dataset)
for prompt in processed_dataset:
    if prompt:
         all_prompts.append({'text': prompt})
dataset = Dataset.from_list(all_prompts)
print(f"Processed dataset has {len(dataset)} examples")

# Free up GPU memory
print("Moving Whisper model to CPU")
data_processor.whisper_model.to('cpu')
torch.cuda.empty_cache()

Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loading Whisper model: large-v2 on cuda
Whisper model loaded.
Starting Arabic dataset processing...


Processing Arabic Dataset:   0%|          | 0/50 [00:00<?, ?it/s]

Processing Arabic text: اترككم مع الحوار حياك الله ابو...


...

Processing Arabic text: الى نهاية الالفين وعشرة



Processing Arabic Dataset:  98%|█████████▊| 49/50 [02:49<00:02,  2.52s/it]

Processing Arabic text: وقبلها كنت فالبنك الهولندي وقب...


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")

Processing Arabic Dataset: 100%|██████████| 50/50 [02:53<00:00,  3.46s/it]


Arabic dataset processing finished. Processed: 49, Skipped: 1
Processed dataset has 49 examples
Moving Whisper model to CPU


In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Set up the trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 60,  # Increase for better results
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

# Show memory stats before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Run training
trainer_stats = trainer.train()

# Show memory stats after training
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/49 [00:00<?, ? examples/s]

GPU = Tesla T4. Max memory = 14.741 GB.
11.352 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 49 | Num Epochs = 10 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488/1,262,028,800 (1.08% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.018400
2,3.590400
3,4.070900
4,3.863800
5,3.863500
6,3.905700
7,3.667600
8,3.796300
9,3.721600
10,3.843600


248.8548 seconds used for training.
4.15 minutes used for training.
Peak reserved memory = 11.352 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 77.01 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [9]:
def generate_arabic_tts(input_text):
    """Generate TTS for Arabic text using OuteTTS's native Arabic support"""
    import re
    import torch
    import numpy as np
    from transformers import LogitsProcessor
    import transformers.generation.utils as generation_utils

    # Use the Arabic text directly - OuteTTS has native Arabic support!
    print(f"Generating speech for: {input_text}")

    # Use the correct repetition penalty implementation from the model card
    class RepetitionPenaltyLogitsProcessorPatch(LogitsProcessor):
        def __init__(self, penalty: float):
            penalty_last_n = 64  # Important: This model needs 64-token window
            print(f"🔄 Using patched RepetitionPenaltyLogitsProcessor | penalty_last_n: {penalty_last_n}")
            self.penalty_last_n = penalty_last_n
            self.penalty = penalty

        @torch.no_grad()
        def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
            if self.penalty_last_n == 0 or self.penalty == 1.0:
                return scores

            batch_size, seq_len = input_ids.shape
            vocab_size = scores.shape[-1]

            for b in range(batch_size):
                start_index = max(0, seq_len - self.penalty_last_n)
                window_indices = input_ids[b, start_index:]

                if window_indices.numel() == 0:
                    continue

                tokens_in_window = set(window_indices.tolist())

                for token_id in tokens_in_window:
                    if token_id >= vocab_size:
                        continue

                    logit = scores[b, token_id]

                    if logit <= 0:
                        logit *= self.penalty
                    else:
                        logit /= self.penalty

                    scores[b, token_id] = logit

            return scores

    generation_utils.RepetitionPenaltyLogitsProcessor = RepetitionPenaltyLogitsProcessorPatch

    # Function to extract audio tokens from output
    def get_audio(tokens):
        decoded_output = tokenizer.batch_decode(tokens, skip_special_tokens=False)[0]
        c1 = list(map(int, re.findall(r"<\|c1_(\d+)\|>", decoded_output)))
        c2 = list(map(int, re.findall(r"<\|c2_(\d+)\|>", decoded_output)))

        t = min(len(c1), len(c2))
        c1 = c1[:t]
        c2 = c2[:t]
        output = [c1, c2]
        if not output or len(output[0]) == 0:
            print("No audio tokens found in the output")
            return None

        return data_processor.audio_processor.audio_codec.decode(
            torch.tensor([output], dtype=torch.int64).to(data_processor.audio_processor.audio_codec.device)
        )

    # Format text for model using direct Arabic text
    formated_text = "<|text_start|>" + input_text + "<|text_end|>"
    prompt = "\n".join([
        "<|im_start|>",
        formated_text,
        "<|audio_start|><|global_features_start|>",
    ])

    # Check tokenization
    input_tokens = tokenizer.tokenize(prompt)
    print(f"Prompt token count: {len(input_tokens)}")

    # Generate audio tokens with the recommended sampling settings from the model card
    with torch.inference_mode():
        with torch.amp.autocast('cuda', dtype=model.dtype):
            model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

            print("Generating token sequence for Arabic text...")
            generated_ids = model.generate(
                **model_inputs,
                temperature=0.4,       # Recommended value
                top_k=40,              # Recommended value
                top_p=0.9,             # Recommended value
                repetition_penalty=1.1, # Recommended value
                min_p=0.05,            # Recommended value
                max_new_tokens=2048,
            )
            print("Token sequence generated.")

    generated_ids_trimmed = generated_ids[:, model_inputs.input_ids.shape[1]:]

    # Debug info
    gen_tokens = tokenizer.convert_ids_to_tokens(generated_ids_trimmed[0])
    audio_token_count = sum(1 for t in gen_tokens if "<|c" in t)
    print(f"Generated {audio_token_count} audio tokens")

    # Get audio
    audio = get_audio(generated_ids_trimmed)

    if audio is not None:
        audio = audio.cpu()
        from IPython.display import Audio as IPythonAudio
        return IPythonAudio(audio.squeeze(0), rate=24000)
    else:
        print("Failed to generate audio")
        return None

In [10]:
# Test with different Arabic phrases
test_phrases = [
    "اسمي عبدالله خان",  # "My name is Abdullah Khan"
    "مرحبا بكم في المملكة العربية السعودية",  # "Welcome to Saudi Arabia"
    "كيف حالك اليوم؟"  # "How are you today?"
]

# Test each phrase
for phrase in test_phrases:
    print(f"\n=== Testing: {phrase} ===")
    audio = generate_arabic_tts(phrase)
    display(audio)


=== Testing: اسمي عبدالله خان ===
Generating speech for: اسمي عبدالله خان
Prompt token count: 11
Generating token sequence for Arabic text...
🔄 Using patched RepetitionPenaltyLogitsProcessor | penalty_last_n: 64
Token sequence generated.
Generated 355 audio tokens



=== Testing: مرحبا بكم في المملكة العربية السعودية ===
Generating speech for: مرحبا بكم في المملكة العربية السعودية
Prompt token count: 16
Generating token sequence for Arabic text...
🔄 Using patched RepetitionPenaltyLogitsProcessor | penalty_last_n: 64
Token sequence generated.
Generated 414 audio tokens


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")



=== Testing: كيف حالك اليوم؟ ===
Generating speech for: كيف حالك اليوم؟
Prompt token count: 13
Generating token sequence for Arabic text...
🔄 Using patched RepetitionPenaltyLogitsProcessor | penalty_last_n: 64
Token sequence generated.
Generated 1174 audio tokens


In [11]:
# Save the fine-tuned model locally
model.save_pretrained("arabic_oute_tts_lora")
tokenizer.save_pretrained("arabic_oute_tts_lora")
print("Model saved locally to 'arabic_oute_tts_lora'")

# Push to Hugging Face Hub (uncomment and add your token if needed)
# model.push_to_hub("YOUR-USERNAME/arabic-oute-tts-lora", token="YOUR-HF-TOKEN")
# tokenizer.push_to_hub("YOUR-USERNAME/arabic-oute-tts-lora", token="YOUR-HF-TOKEN")

Model saved locally to 'arabic_oute_tts_lora'


In [12]:
def speak_arabic(text):
    """
    A simplified function to convert Arabic text to speech
    """
    print(f"Converting to speech: {text}")
    audio = generate_arabic_tts(text)
    return audio

# Try with a custom phrase
custom_text = "أهلا وسهلا بك في العالم العربي"  # "Welcome to the Arab world"
speak_arabic(custom_text)

Converting to speech: أهلا وسهلا بك في العالم العربي
Generating speech for: أهلا وسهلا بك في العالم العربي
Prompt token count: 17
Generating token sequence for Arabic text...
🔄 Using patched RepetitionPenaltyLogitsProcessor | penalty_last_n: 64
Token sequence generated.
Generated 339 audio tokens


In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [ ]:
tokenizer.tokenize("اسمي عبدالله خان، أعيش في المملكة العربية السعودية")


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-da")
tokenizer.tokenize("مرحبا بكم في عالم الذكاء الاصطناعي")


And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
